# Phase 1: Analysis & Scope (Gemini Powered)

이 노트는 Gemini API를 사용하여 **Phase 1: 기획 및 범위 설정** 단계를 구현합니다.
구조화된 출력(JSON Schema)을 사용하여 에이전트가 항상 예측 가능한 형식으로 응답하도록 강제합니다.

## 1. Environment & API Key Setup
Google Gemini API 키를 설정합니다. 아래 변수에 키를 입력하세요.

In [ ]:
import os
import json
import time
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content

# TODO: 여기에 API Key를 입력하세요
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)

## 2. JSON Schema Definition
Planning Agent가 출력해야 할 JSON 구조를 정의합니다.

In [20]:
planning_schema = {
  "type": "object",
  "properties": {
    "has_missing_info": {
      "type": "boolean"
    },
    "missing_info_list": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "draft_plan": {
      "type": "object",
      "properties": {
        "project_meta": {
          "type": "object",
          "properties": {
            "title": {
              "type": "string"
            },
            "target_audience": {
              "type": "string"
            },
            "goal": {
              "type": "string"
            }
          },
          "required": [
            "title",
            "target_audience",
            "goal"
          ]
        },
        "style_guide": {
          "type": "object",
          "properties": {
            "tone": {
              "type": "string"
            },
            "detail_level": {
              "type": "string",
              "nullable": True
            },
            "math_policy": {
              "type": "string"
            },
            "example_policy": {
              "type": "string"
            },
            "formatting": {
              "type": "string"
            }
          },
          "required": [
            "tone",
            "detail_level",
            "math_policy",
            "example_policy",
            "formatting"
          ]
        },
        "chapters": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "id": {
                "type": "integer"
              },
              "title": {
                "type": "string"
              },
              "objective": {
                "type": "string"
              },
              "key_topics": {
                "type": "array",
                "items": {
                  "type": "string"
                }
              },
              "must_include": {
                "type": "array",
                "items": {
                  "type": "string"
                }
              }
            },
            "required": [
              "id",
              "title",
              "objective",
              "key_topics",
              "must_include"
            ]
          }
        }
      },
      "required": [
        "project_meta",
        "style_guide",
        "chapters"
      ]
    }
  },
  "required": [
    "has_missing_info",
    "missing_info_list",
    "draft_plan"
  ]
}

## 3. System Prompt
Planning Agent의 역할과 규칙을 정의하는 시스템 프롬프트입니다.

In [21]:
SYSTEM_PROMPT_TEXT = """
You are the **Planning Agent** for a Lecture Material Generation System.
Your goal is to construct or update a structured `Draft Plan` based on user input.

### **Input Context**
You will receive:
1. **User Input**: The user's latest message.
2. **Current Draft Plan** (Optional): The JSON object from the previous turn.
3. **Force Completion Mode** (Boolean): If `true`, you must finalize the plan immediately.

### **Execution Logic**
1. **Load Context**:
   - Use **Current Draft Plan** as the baseline if provided. Otherwise, start blank.

2. **Analyze & Update**:
   - Analyze **User Input** to extract explicit information.
   - **FILL** only the fields that were previously `null` or requested to be changed.
   - **PRESERVE** existing data unless asked to change.

3. **Check Conditions (The Fork)**:
   - **Scenario A: Normal Mode (`Force Mode` == `false`)**
     - Strictly follow **NULL POLICY**. Do NOT guess.
     - If critical info (Target, Goal) is missing, keep it `null` and set `"has_missing_info": true`.
   
   - **Scenario B: Force Mode (`Force Mode` == `true`)**
     - **OVERRIDE NULL POLICY**.
     - You MUST fill all remaining `null` fields with **reasonable defaults** based on context.
     - You MUST set `"has_missing_info": false`.
     - You MUST return an empty `"missing_info_list"`.

4. **Structure Chapters**: 
   - Generate or refine up to **10 chapters** logically to create a complete curriculum.

### **Critical Rules (STRICT)**
- **NORMAL MODE**: Explicit info only. If vague, keep `null`. Ask again.
- **FORCE MODE**: Do not ask again. Guess reasonably to finalize the plan.
- **OUTPUT LANGUAGE**: Korean.
- **OUTPUT FORMAT**: JSON Only.

### **Few-shot Example (Update with Force Mode Scenario)**
**Input Context**:
- **Current Draft Plan**: { "project_meta": { "title": "엑셀 강의", "target_audience": null, "goal": null }, ... }
- **User Input**: "그냥 알아서 적당히 완성해줘."
- **Force Completion Mode**: **true**

**Agent Output**:
{
  "has_missing_info": false,  // <--- Force Mode active: Must be false
  "missing_info_list": [],    // <--- No more questions allowed
  "draft_plan": {
    "project_meta": {
      "title": "실무에서 바로 쓰는 엑셀 기초",
      "target_audience": "엑셀 입문자 및 초급 직장인",  // <--- AI inferred default
      "goal": "기본적인 데이터 입력부터 간단한 함수 활용까지 마스터한다." // <--- AI inferred default
    },
    "style_guide": {
      "tone": "친절하고 전문적인",
      "detail_level": "초급",
      "math_policy": "최소화",
      "example_policy": "비즈니스 예제 위주",
      "formatting": "기본"
    },
    "chapters": [
      {
        "id": 1,
        "title": "엑셀과 친해지기",
        "objective": "엑셀 인터페이스를 이해하고 셀 선택 및 입력을 익힌다.",
        "key_topics": ["화면 구성", "데이터 입력", "저장하기"],
        "must_include": ["단축키 모음"]
      }
      // ... (Generates up to 10 chapters to complete the plan)
    ]
  }
}
"""

## 4. Planning Agent Implementation
Gemini 모델을 호출하여 계획을 생성하는 클래스입니다.

In [22]:
class PlanningAgent:
    def __init__(self, model_name="gemini-3-flash-preview"):
        # 구조화된 출력을 위한 설정
        self.model = genai.GenerativeModel(
            model_name=model_name,
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": planning_schema
            },
            system_instruction=SYSTEM_PROMPT_TEXT
        )
        self.current_draft = None

    def generate_plan(self, user_input, force_completion=False):
        """
        user_input: 사용자의 최신 입력
        force_completion: 강제 완성 모드 활성화 여부
        """
        # 프롬프트 구성
        if self.current_draft is None:
            context_str = "Current Draft Plan: None (Start fresh)"
        else:
            # 이전 초안을 JSON 문자열로 변환하여 프롬프트에 포함
            context_str = f"Current Draft Plan: {json.dumps(self.current_draft, ensure_ascii=False)}"

        prompt = f"""
        [Context Info]
        {context_str}
        
        [Current User Input]
        {user_input}
        
        [Mode]
        Force Completion Mode: {force_completion}
        """

        print(f"\n[Agent] Gemini에게 요청을 보냅니다... (Force Mode: {force_completion})")
        try:
            response = self.model.generate_content(prompt)
            
            # 응답 텍스트를 JSON으로 파싱
            response_json = json.loads(response.text)
            
            # 내부 상태 업데이트 (최신 초안 저장)
            if "draft_plan" in response_json:
                 self.current_draft = response_json["draft_plan"]
            
            return AgentResponse(
                has_missing_info=response_json.get("has_missing_info", False),
                missing_info_list=response_json.get("missing_info_list", []),
                draft_plan=response_json.get("draft_plan", {})
            )
            
        except Exception as e:
            print(f"\n[Error] API 호출 중 오류 발생: {e}")
            return None

class AgentResponse:
    def __init__(self, has_missing_info, missing_info_list, draft_plan):
        self.has_missing_info = has_missing_info
        self.missing_info_list = missing_info_list
        self.draft_plan = draft_plan

## 5. Phase 1 Execution Logic
사용자와 상호작용하며 기획을 완성하는 메인 로직입니다.

In [23]:
def Execute_Phase1(model_name="gemini-3-flash-preview"):
    import sys
    import time

    # 1. 초기화
    agent = PlanningAgent(model_name=model_name)
    MAX_RETRY_LIMIT = 3
    current_retry = 0
    
    # 최초 사용자 입력 받기
    print("\n[Step 1] 만들고 싶은 강의의 주제나 요구사항을 입력해주세요:")
    user_input = input(">>> ")

    # 2. 기획 수립 루프
    while True:
        # Retry 횟수가 초과되면 강제 완성 모드 활성화
        is_force_mode = (current_retry >= MAX_RETRY_LIMIT)
        
        # Agent 호출
        agent_response = agent.generate_plan(user_input, force_completion=is_force_mode)
        
        if not agent_response:
            print("Agent 응답 실패. 다시 시도해주세요.")
            continue

        # Case A: 정보 부족 (질문 수행)
        if agent_response.has_missing_info and not is_force_mode:
            print("\n[System] 더 정확한 기획을 위해 추가 정보가 필요합니다.")
            
            # 질문 출력
            for i, q in enumerate(agent_response.missing_info_list, 1):
                print(f"System: {q}")
            
            # 출력 버퍼 비우기 및 잠시 대기 (Input이 출력을 가리지 않도록)
            sys.stdout.flush()
            time.sleep(0.5)
            
            # 사용자 답변 입력
            user_answer = input("\n위 질문에 대한 답변을 입력해주세요: ")
            
            # 사용자 입력을 업데이트 (질문 + 답변 형태로 전달하여 맥락 유지)
            user_input = f"이전 질문에 대한 사용자의 답변: {user_answer}"
            current_retry += 1
            
        # Case B: 기획 완료
        else:
            return agent_response.draft_plan

## 6. Run Application

In [24]:
if __name__ == "__main__":
    print("===========================================")
    print("   Lecture Content Generator - Phase 1")
    print("===========================================")
    
    try:
        # 실행
        final_plan = Execute_Phase1()
        
        # 최종 결과 출력
        print("\n" + "="*50)
        print(" [Final Output] Generated Draft Plan")
        print("="*50)
        print(json.dumps(final_plan, indent=2, ensure_ascii=False))
        print("="*50)
        
    except KeyboardInterrupt:
        print("\n[System] 사용자에 의해 중단되었습니다.")

   Lecture Content Generator - Phase 1

[Step 1] 만들고 싶은 강의의 주제나 요구사항을 입력해주세요:

[Agent] Gemini에게 요청을 보냅니다... (Force Mode: False)

[System] 더 정확한 기획을 위해 추가 정보가 필요합니다.
System: target_audience (수강 대상)
System: goal (강의의 목표)
System: style_guide (강의의 톤앤매너, 상세 정도, 수식 및 예시 활용 정책)

[Agent] Gemini에게 요청을 보냅니다... (Force Mode: False)

[System] 더 정확한 기획을 위해 추가 정보가 필요합니다.
System: 강의 자료의 포맷팅 스타일(예: 불렛포인트 중심, 논문 스타일, 강의 슬라이드 형식 등)에 대한 구체적인 요청 사항이 있으신가요?

[Agent] Gemini에게 요청을 보냅니다... (Force Mode: False)

 [Final Output] Generated Draft Plan
{
  "project_meta": {
    "title": "머신러닝의 기초",
    "target_audience": "대학교 1학년",
    "goal": "머신러닝 핵심 원리 습득 및 딥러닝 심화 학습을 위한 수학적·이론적 기반 구축"
  },
  "style_guide": {
    "tone": "객관적이고 엄밀한 학술 논문 스타일의 문체",
    "detail_level": "대학교 1학년 수준 (수학적 전개는 전문가 수준)",
    "math_policy": "전문가 수준의 엄밀한 수식 증명 및 전개 포함",
    "example_policy": "수식을 활용한 직접적인 계산 예시 및 이론적 사례 중심",
    "formatting": "학술 논문 표준 규격 (Abstract, Introduction, Methodology, Conclusion) 및 인용 체계 적용"
  },
  "chapters": [
 

==================================================
 [Final Output] Generated Draft Plan
==================================================
{
  "project_meta": {
    "title": "머신러닝의 기초",
    "target_audience": "대학교 1학년",
    "goal": "머신러닝 핵심 원리 습득 및 딥러닝 심화 학습을 위한 수학적·이론적 기반 구축"
  },
  "style_guide": {
    "tone": "객관적이고 엄밀한 학술 논문 스타일의 문체",
    "detail_level": "대학교 1학년 수준 (수학적 전개는 전문가 수준)",
    "math_policy": "전문가 수준의 엄밀한 수식 증명 및 전개 포함",
    "example_policy": "수식을 활용한 직접적인 계산 예시 및 이론적 사례 중심",
    "formatting": "학술 논문 표준 규격 (Abstract, Introduction, Methodology, Conclusion) 및 인용 체계 적용"
  },
  "chapters": [
    {
      "id": 1,
      "title": "머신러닝 개요 및 수학적 기초",
      "objective": "머신러닝의 정의와 선형대수학, 미분 등 기초 수학 지식을 복습한다.",
      "key_topics": [
        "지도학습 vs 비지도학습",
        "벡터와 행렬 연산",
        "편미분과 연쇄 법칙"
      ],
      "must_include": [
        "손실 함수의 수학적 정의"
      ]
    },
    {
      "id": 2,
      "title": "선형 회귀 (Linear Regression) 분석",
      "objective": "최소제곱법을 통한 선형 회귀의 수학적 도출 과정을 이해한다.",
      "key_topics": [
        "가설 함수(Hypothesis)",
        "비용 함수(Cost Function)",
        "정규 방정식(Normal Equation)"
      ],
      "must_include": [
        "행렬 미분을 이용한 파라미터 최적화 증명"
      ]
    },
    {
      "id": 3,
      "title": "경사하강법 (Gradient Descent) 최적화",
      "objective": "함수의 최솟값을 찾기 위한 반복적 최적화 알고리즘을 학습한다.",
      "key_topics": [
        "학습률(Learning Rate)",
        "전역 최적점과 지역 최적점",
        "Batch vs SGD vs Mini-batch"
      ],
      "must_include": [
        "경사하강법의 수렴 조건 및 계산 예시"
      ]
    },
    {
      "id": 4,
      "title": "로지스틱 회귀와 분류 이론",
      "objective": "이진 분류를 위한 확률론적 모델의 수식을 이해한다.",
      "key_topics": [
        "시그모이드 함수(Sigmoid Function)",
        "결정 경계(Decision Boundary)",
        "오즈비(Odds Ratio)"
      ],
      "must_include": [
        "최대우도법(MLE)을 통한 로스 유도"
      ]
    },
    {
      "id": 5,
      "title": "교차 엔트로피와 정보 이론",
      "objective": "분류 문제에서 사용되는 손실 함수의 정보이론적 배경을 학습한다.",
      "key_topics": [
        "엔트로피(Entropy)",
        "KL 다이버전스",
        "Cross-Entropy Loss"
      ],
      "must_include": [
        "Softmax 함수의 미분 및 역전파 기초"
      ]
    },
    {
      "id": 6,
      "title": "모델 평가 지표와 검증",
      "objective": "학습된 모델의 성능을 정량적으로 평가하는 방법을 익힌다.",
      "key_topics": [
        "Confusion Matrix",
        "Precision, Recall, F1-score",
        "ROC-AUC 곡선"
      ],
      "must_include": [
        "각 지표간의 트레이드오프 수식 분석"
      ]
    },
    {
      "id": 7,
      "title": "과적합 방지와 규제화 (Regularization)",
      "objective": "모델의 일반화 성능을 높이기 위한 수학적 기법을 배운다.",
      "key_topics": [
        "Bias-Variance Tradeoff",
        "L1 규제 (Lasso)",
        "L2 규제 (Ridge)"
      ],
      "must_include": [
        "라그랑주 승수법을 이용한 규제항의 해석"
      ]
    },
    {
      "id": 8,
      "title": "인공 신경망의 기초 (Perceptron)",
      "objective": "딥러닝의 기본 단위인 퍼셉트론의 구조와 한계를 이해한다.",
      "key_topics": [
        "단층 퍼셉트론",
        "XOR 문제와 비선형성",
        "활성화 함수(Activation Functions)"
      ],
      "must_include": [
        "퍼셉트론 수렴 정리"
      ]
    },
    {
      "id": 9,
      "title": "오차 역전파 (Backpropagation) 알고리즘",
      "objective": "다층 신경망을 학습시키기 위한 핵심 알고리즘의 수식을 전개한다.",
      "key_topics": [
        "Computational Graph",
        "Chain Rule 전개",
        "Gradient Flow"
      ],
      "must_include": [
        "다층 퍼셉트론에서의 가중치 업데이트 수식 유도"
      ]
    },
    {
      "id": 10,
      "title": "딥러닝으로의 확장",
      "objective": "머신러닝 기초를 바탕으로 심화 딥러닝 연구 방향을 제시한다.",
      "key_topics": [
        "심층 신경망(DNN) 구조",
        "기울기 소실 문제(Vanishing Gradient)",
        "향후 학습 로드맵"
      ],
      "must_include": [
        "현대적 딥러닝 프레임워크와 기초 모델 비교"
      ]
    }
  ]
}
==================================================